In [4]:
import re
import time
import json
import csv
import unicodedata
import openai
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from file_save import *

In [19]:
def compute_accuracy(tuples):
    """Measure the GPT3.5 model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        gpt_choices = t["Llama2_choices_pure"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))

    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


In [10]:
folder_path = 'C:/Users/User1/Desktop/master/dissertation/Model_test/chatGPT/result-temp/result-Llama2/'
file_paths = glob.glob(folder_path + '*')
file_paths[0].split("\\")[-1]

'en-ar'

In [30]:
folder_path = 'C:/Users/User1/Desktop/master/dissertation/Model_test/chatGPT/result-temp/result-Llama2/'
PRETICATES = ["P1412", "P1376", "P1303", "P530", "P495", "P449", "P364", "P264", "P190", "P136", "P106", "P103", "P47", "P37", "P36", "P30", "P27", "P20", "P19", "P17"]

# Get a list of all file paths in the folder
file_paths = glob.glob(folder_path + '*')
df_info = {}

lang = ["en-extend", "en-es", "en-ar"]
model_name_list = ["llama"]
file_name_list = []
total_list = []
for i in lang:
    info = {}
    info["lang_list"] = []
    info["N_c_e"] = []
    info["P_c_e"] = []
    info["N_c_o"] = []
    info["P_c_o"] = []
    info["time_cost"] = []

    df_info[i] = info
# Iterate over the file paths and read each file
count = 0
for file_path in file_paths:
    current_lang = file_path.split("\\")[-1]
    
    file_pathss = glob.glob(file_path + '/*')
    for input_file_path in file_pathss:
        start = time.perf_counter()
        output_triple = load_jsonl_file(input_file_path)
        current_file_name = input_file_path.split("\\")[-1]
        
        current_file_name= current_file_name.replace(".jsonl", "")
        current_result_list = list(compute_accuracy(output_triple).values())
        
        file_name_list.append(current_file_name)
        total_list.append(current_result_list[0])

        end = time.perf_counter()
        elapsed = end - start
        df_info[current_lang]["lang_list"].append(current_lang)
        df_info[current_lang]["N_c_e"].append(current_result_list[1])
        df_info[current_lang]["P_c_e"].append(current_result_list[2])
        df_info[current_lang]["N_c_o"].append(current_result_list[3])
        df_info[current_lang]["P_c_o"].append(current_result_list[4])
        df_info[current_lang]["time_cost"].append(elapsed)
        
        columns = ['model name','file name','tuples numbers']
        data = [model_name_list,file_name_list, total_list]
for ii in lang:
    columns.extend(['language','e', 'e%', 'o', 'o%', "time cost"])
    data.append(df_info[ii]["lang_list"])
    data.append(df_info[ii]["N_c_e"])
    data.append(df_info[ii]["P_c_e"])
    data.append(df_info[ii]["N_c_o"])
    data.append(df_info[ii]["P_c_o"])
    data.append(df_info[ii]["time_cost"])
data = tuple(data)

df = pd.DataFrame(data,index=columns).T
df.to_excel("test.xlsx")
        
    
    

In [26]:
load_jsonl_file(input_file_path)

[{'sub_uri': 'Q39',
  'obj_uri': ['Q1016',
   'Q1246',
   'Q142',
   'Q145',
   'Q148',
   'Q159',
   'Q159583',
   'Q16',
   'Q183',
   'Q184',
   'Q217',
   'Q218',
   'Q227',
   'Q252',
   'Q258',
   'Q28',
   'Q30',
   'Q347',
   'Q35',
   'Q38',
   'Q399',
   'Q40',
   'Q403',
   'Q408',
   'Q41',
   'Q414',
   'Q423',
   'Q43',
   'Q458',
   'Q77',
   'Q79',
   'Q794',
   'Q801',
   'Q833',
   'Q843',
   'Q854',
   'Q865',
   'Q884',
   'Q928',
   'Q96'],
  'sub_label': 'Switzerland',
  'obj_label': ['ID',
   'Taiwan',
   'Chunghwa Minkwo',
   'State of Libya',
   'el',
   'Egypt',
   'United States of America',
   'France',
   'European Union',
   'Bielaruś',
   'md',
   'Republic of China (Taiwan)',
   'Federation of Malaysia',
   '🇱🇾',
   'az',
   'Hellas',
   'Belarus',
   'Chair of Peter',
   'the US of America',
   'Taprobane',
   'Europe',
   'Korea (South)',
   'N. Korea',
   'eu',
   'Azərbaycan Respublikası',
   'Rep. Egypt',
   'id',
   '🇱🇮',
   'U. S. A.',
   'US of M

In [ ]:
input_file_path = "dlama-v1\en-es\P449_general_WESTERN_COUNTRIES_output.jsonl"
tuples = load_jsonl_file(input_file_path)
print(compute_accuracy(tuples))